In [19]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("emineyetm/fake-news-detection-datasets")

print("Path to dataset files:", path)

Path to dataset files: /Users/najiaraji/.cache/kagglehub/datasets/emineyetm/fake-news-detection-datasets/versions/1


In [20]:
import pandas as pd

# Charger un fichier CSV
df_fake = pd.read_csv('/Users/najiaraji/Downloads/News _dataset/Fake.csv')
df_true = pd.read_csv('/Users/najiaraji/Downloads/News _dataset/True.csv')
print(df_fake.head())

                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  


In [ ]:
import numpy as np
import pandas as pd
import sklearn
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

In [ ]:
#Librairies
import matplotlib.pyplot as plt
import tensorflow as tensorflow
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.utils import pad_sequences
import nltk
from nltk.stem.snowball import SnowballStemmer
import regex as re
from nltk.tokenize import sent_tokenize
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords


In [23]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/najiaraji/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/najiaraji/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/najiaraji/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [25]:
df_fake.head()


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
# label them seperately
df_true['status'] = 0
df_fake['status'] = 1

In [28]:
# merge and remove unnecessary columns
df = pd.concat([df_true,df_fake])
df.drop(['subject','text','date'],axis=1,inplace=True)
df

,title,status
0,"As U.S. budget fight looms, Republicans flip t...",0
1,U.S. military to accept transgender recruits o...,0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,0
3,FBI Russia probe helped by Australian diplomat...,0
4,Trump wants Postal Service to charge 'much mor...,0
...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,1
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,1
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,1
23479,How to Blow $700 Million: Al Jazeera America F...,1


In [29]:
# let's blend the smoothie
random_indexes = np.random.randint(0,len(df),len(df))
df = df.iloc[random_indexes].reset_index(drop=True)
df

,title,status
0,NAACP CHIEF Asks BLM Rioters To “Show up en ma...,1
1,TV star urges protests as his lead shrinks in ...,0
2,BLACK JUDGE FINDS White Cop In Freddie Gray Ca...,1
3,Magnitude 7.0 undersea quake hits near New Cal...,0
4,CLUELESS ANTI-TRUMP PROTESTERS Asked Why They ...,1
...,...,...
44893,BITTER HILLARY JUST CLAIMED SHE ‘BEAT’ TRUMP: ...,1
44894,Parents Ban Black History Month Video From Sc...,1
44895,UNHINGED MIKA Called President Trump “Not Well...,1
44896,"Trump Doubles Down, Says Obama LITERALLY Crea...",1


In [30]:
#text analysis
pd.set_option('display.max_colwidth', 500)
random = np.random.randint(0,len(df),20)
df.iloc[random]


,title,status
1488,SpaceX: Explosion Rocks Launchpad at Firm’s Cape Canaveral Facility in Florida,1
29205,JUST IN: PRESIDENT TRUMP BLASTS “Retiring” FBI Deputy Director Andrew McCabe and “’Leakin’ James Comey” After JUDICIAL WATCH Uncovers A McCabe Bombshell,1
28296,BARBRA STREISAND Gives Up On Dream Of Impeaching Trump Over Fake Trump-Russian Collusion…Tweets HILARIOUS New Reason Trump Should Be Impeached,1
15391,STUNNING! FORMER DISNEY TECH WORKER’S Emotional Testimony Of Humiliation In Training Foreign Workers [Video],1
34929,WATCH: Keith Olbermann Tells Fools Who Voted For Trump To Wave Goodbye To Their Social Security,1
3111,Mueller Team Uniform? ‘Democratic Donkey Jerseys’ and ‘I’m With Hillary T-Shirts’ says Congressman,1
21394,NZ First leader says foreign ownership to be part of coalition wrangling,0
17617,WOW! Watch Former Democrat Hammer Obama On His Weak Leadership And Hatred For America,1
13857,EMBARRASSING: Anti-Trump “THE HILL” Gets SLAMMED For Publishing Article Claiming MTV’s “Jersey Shore” Reality Star “Schooled” Trump On Global Warming,1
35871,“MODERATE” MUSLIM SILENCE? Neighbors: “They Kept To Themselves…They Did Not Wear Religious Garb”…PEOPLE WHO KNEW HIM: “He Expressed Support For Terror Groups”,1


In [31]:
# Null values
df.isnull().sum()

title     0
status    0
dtype: int64

In [ ]:
# longest sentence length
def longest_sentence_length(text):
  return len(text.split())

df['maximum_length'] = df['title'].apply(lambda x : longest_sentence_length(x))
print('longest sentence having length -')
max_length = max(df['maximum_length'].values)
print(max_length)

longest sentence having length -
42


In [33]:
# Text cleaning
text_cleaning = "\b0\S*|\b[^A-Za-z0-9]+"

def preprocess_filter(text, stem=False):
  text = re.sub(text_cleaning, " ",str(text.lower()).strip())
  tokens = []
  for token in text.split():
    if token not in stop_words:
      if stem:
        stemmer = SnowballStemmer(language='english')
        token = stemmer.stem(token)
      tokens.append(token)
  return " ".join(tokens)

In [ ]:
# Word embedding with pre padding
def one_hot_encoded(text,vocab_size=5000,max_length = 40):
    hot_encoded = one_hot(text,vocab_size)
    return hot_encoded

In [35]:
# word embedding pipeline
def word_embedding(text):
    preprocessed_text=preprocess_filter(text)
    return one_hot_encoded(preprocessed_text)

In [36]:
# Creating NN Model
embedded_features = 40
model = Sequential()
model.add(Embedding(5000,embedded_features,input_length = max_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer= 'adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [37]:
# One hot encoded title
one_hot_encoded_title =df['title'].apply(lambda x : word_embedding(x)).values

In [38]:
# padding to make the size equal of the sequences
padded_encoded_title = pad_sequences(one_hot_encoded_title,maxlen=max_length,padding = 'pre')

In [ ]:
# Splitting
X = padded_encoded_title
y = df['status'].values
y = np.array(y)

# shapes
print(X.shape)
print(y.shape)

(44898, 42)
(44898,)


In [40]:
# shape and size
print('X shape {}'.format(X.shape))
print('y shape {}'.format(y.shape))

X shape (44898, 42)
y shape (44898,)


In [41]:
# Splitting into training, testing
X_train,X_test,y_train,y_test = train_test_split(X,y, random_state = 42)

# Shape and size of train and test dataset
print('X train shape {}'.format(X_train.shape))
print('X test shape {}'.format(X_test.shape))
print('y train shape {}'.format(y_train.shape))
print('y test shape {}'.format(y_test.shape))

X train shape (33673, 42)
X test shape (11225, 42)
y train shape (33673,)
y test shape (11225,)


In [43]:
# Model training
# training
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=64)

Epoch 1/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 40s 76ms/step - accuracy: 0.9993 - loss: 0.0025 - val_accuracy: 0.9686 - val_loss: 0.2172
Epoch 2/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 52s 98ms/step - accuracy: 0.9995 - loss: 0.0013 - val_accuracy: 0.9654 - val_loss: 0.2435
Epoch 3/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 53s 101ms/step - accuracy: 0.9991 - loss: 0.0034 - val_accuracy: 0.9685 - val_loss: 0.1881
Epoch 4/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 50s 95ms/step - accuracy: 0.9998 - loss: 0.0011 - val_accuracy: 0.9691 - val_loss: 0.2112
Epoch 5/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 54s 102ms/step - accuracy: 1.0000 - loss: 1.7400e-04 - val_accuracy: 0.9664 - val_loss: 0.2314
Epoch 6/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 49s 93ms/step - accuracy: 1.0000 - loss: 5.0540e-05 - val_accuracy: 0.9676 - val_loss: 0.2443
Epoch 7/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 48s 90ms/step - accuracy: 1.0000 - loss: 2.5866e-05 - val_accuracy: 0.9678 - val_loss: 0.2537
Epoch 8/15
527/527 ━━━━━━━━━━━━━━━━━━━━ 36s 69ms/step - accuracy: 1.0000 - l

In [58]:
#evaluation
# setting threshold value
def best_threshold_value(thresholds:list,X_test):
    accuracies = []
    for thresh in thresholds:
        ypred =model.predict(X_test)
        ypred = np.where(ypred> thresh,1,0)
        accuracies.append(accuracy_score(y_test,ypred))
    return pd.DataFrame({
        'Threshold': thresholds,
        'Accuracy' : accuracies
    })


In [59]:
best_threshold_value([0.4,0.5,0.6,0.7,0.8,0.9], X_test)

351/351 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step
351/351 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step
351/351 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step
351/351 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step
351/351 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step
351/351 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step


,Threshold,Accuracy
0,0.4,0.968018
1,0.5,0.968463
2,0.6,0.968998
3,0.7,0.969354
4,0.8,0.969532
5,0.9,0.970067


In [60]:
# Prediction value at threshold 0.4
y_pred = model.predict(X_test)
y_pred = np.where(y_pred >0.4, 1, 0)

351/351 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step


In [61]:
# Confusion matrix
print('Confusion matrix')
print(confusion_matrix(y_pred,y_test))
print('----------------')
print('Classification report')
print(classification_report(y_pred,y_test))

Confusion matrix
[[5156  143]
 [ 216 5710]]
----------------
Classification report
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      5299
           1       0.98      0.96      0.97      5926

    accuracy                           0.97     11225
   macro avg       0.97      0.97      0.97     11225
weighted avg       0.97      0.97      0.97     11225



In [71]:
#predictions
# input generator
def prediction_input_processing(text):
    encoded = word_embedding(text)
    padded_encoded_title = pad_sequences([encoded],maxlen=max_length,padding = 'pre')
    output = model.predict(padded_encoded_title)
    output = np.where(0.4>output,1,0)
    if output[0][0] == 1:
        return 'Yes this News is fake'
    return 'No, It is not fake'


In [72]:
# predictions
prediction_input_processing('Americans are more concerned over Indians fake open source contribution')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


'Yes this News is fake'

In [73]:
news = 'Trump Just Sent Michelle Obama a Bill She will Never Be able to pay in her lifetime'
prediction_input_processing(news)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


'No, It is not fake'

In [74]:
import os
print("Répertoire actuel:", os.getcwd())

Répertoire actuel: /Users/najiaraji/code/AurelienCardon/fake_news_detection/notebooks


In [75]:
import os
save_dir = "/Users/najiaraji/code/AurelienCardon/fake_news_detection/notebooks"
os.makedirs(save_dir, exist_ok=True)

model_path = os.path.join(save_dir, "my_model.keras")
model.save(model_path)